In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import sys
sys.path.append('../')
from helpers import *
import plotly.graph_objs as go

### Degree influence in Metric dimension of Erdos_renyi graphs

In [ ]:
# Here we find small resolving set and check the degree of the nodes

# erdos_renyi_graph
n = 300
p = 0.5
nb_of_iters = 100
nb_graphs = 1
lower_limit = 0
number_of_decile = 5

nb_from_each_decile = {}


for _ in tqdm(range(nb_graphs)):
    G = nx.erdos_renyi_graph(n, p)

    node_list = list(G.nodes())

    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_decile
    decile_num = 1
    decile_vertices = []

    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []
    length = dict(nx.all_pairs_shortest_path_length(G))

    for nb in range(lower_limit, G.number_of_nodes()): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                res = nodes
                break
        else:
            continue
        break
        
    second_elements = [t[1] for t in nx.get_node_attributes(G, 'decile').items() if t[0] in res]
    for d in second_elements:
        nb_from_each_decile[d] = nb_from_each_decile.get(d, 0) + 1

In [ ]:
total = sum(nb_from_each_decile.values())

nb_from_each_decile_in_perc = {}
for i in range(1, number_of_decile + 1):
    nb_from_each_decile_in_perc[i] = (nb_from_each_decile[i] / total) * 100

solution_er = dict(sorted(nb_from_each_decile_in_perc.items()))

In [ ]:
# Define your data
x_er = list(solution_er.keys())
y_er = list(solution_er.values())

# Define the trace for the bar plot
trace = go.Bar(x=x_er, y=y_er)

# Define the layout
layout = go.Layout(title='Impact of node degree on Metric Dimension: Analysis by quintiles', 
                   title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Percentage of nodes'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [ ]:
# Here we draw the nodes from the different quintile and 
# check if there are difference in the size of the resolving sets.

solutions_er = {}
nb_of_graphs = 2
number_of_decile = 5

# erdos_renyi_graph
n = 1000
p = 0.5

for g in tqdm(range(nb_of_graphs)):

    G = nx.erdos_renyi_graph(n, p)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    nb_of_iters = 100

    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []


    for d in range(1, number_of_decile+1):       
        nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == d]

        high = n//number_of_decile
        low = 0

        while low + 1 < high:
            middle = (high+low) // 2
            num_nodes = middle # Number of nodes to sample
            node_list = list(G.nodes())
            count = 0
            for i in range(nb_of_iters):
                nodes = set(random.sample(nodes_in_decile, num_nodes)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            v = count / nb_of_iters
            if (count / nb_of_iters) > 0.5:
                high = middle
                v_high = v 
            else:
                low = middle
                v_low = v
        if abs(v_low - 0.5) < abs(v_high - 0.5):
            solutions_er[d] = solutions_er.get(d, 0) + low
        else:
            solutions_er[d] = solutions_er.get(d, 0) + high
            
for i in range(1, number_of_decile+1):
    solutions_er[i] = solutions_er[i] / nb_of_graphs

In [ ]:
# Define your data
x_er = list(solutions_er.keys())
y_er = list(solutions_er.values())

# Define the trace for the bar plot
trace = go.Bar(x=x_er, y=y_er)

# Define the layout
layout = go.Layout(#title='Impact of node degree on Metric Dimension: Analysis by quintiles', 
                   #title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Metric Dimension'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [ ]:
# Here we draw the nodes from the different quintile and 
# check if there are difference in the size of the resolving sets.

# We plot the size of the resolving set in a box plot
solutions_er = {}
nb_of_graphs = 1
number_of_decile = 5
nb_of_iters = 100


# erdos_renyi_graph
n = 1000
p = 0.5

for g in tqdm(range(nb_of_graphs)):

    G = nx.erdos_renyi_graph(n, p)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())

    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []


    for d in range(1, number_of_decile+1): 
        sol = []
        nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == d]

        for _ in range(nb_of_iters):

            nodes = list(np.random.choice(nodes_in_decile, size=len(nodes_in_decile), replace=False)) # Random set of nodes to test
            for nb in range(0, len(nodes_in_decile)+1):
                if is_resolving_set(G, nodes[:nb], length):
                    sol.append(nb)
                    break
        solutions_er[d] = solutions_er.get(d, []) + sol

In [ ]:
fig = go.Figure()

for group, values in solutions_er.items():
    fig.add_trace(go.Box(y=values, name=group))

    
fig.update_layout(
    yaxis=dict(
        title="Size of the resolving set",
        titlefont=dict(size=12, color='black')
    )
)
fig.show()

### Betweenness centrality influence in metric dimension in Erdos_renyi graphs

In [ ]:
# Here we draw the nodes from the different quintile based on the betweenness centrality and 
# check if there are difference in the size of the resolving sets.

nb_of_graphs = 1
nb_decile = 5

# erdos_renyi_graph
n = 300
p = 0.5

solutions_er_bc = {}
for g in tqdm(range(nb_of_graphs)):

    G = nx.erdos_renyi_graph(n, p)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    nb_of_iters = 100

    # Get all shortest paths between every pair of vertices
    shortest_paths = dict(nx.all_pairs_shortest_path(G))

    # Initialize a dictionary to store the counts of shortest paths that pass through each vertex
    path_counts = {v: 0 for v in G.nodes}

    # Count how many of the shortest paths pass through each vertex
    for source in G.nodes:
        for target in G.nodes:
            if source != target:
                paths = shortest_paths[source][target]
                for vertex in paths[1:-1]:
                    path_counts[vertex] += 1

    
    degree_list = [(n, d) for n, d in path_counts.items()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // nb_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []

    for d in range(1, nb_decile+1):       
        nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == d]

        high = n // nb_decile
        low = 0

        while low + 1 < high:
            middle = (high+low) // 2
            num_nodes = middle # Number of nodes to sample
            node_list = list(G.nodes())
            count = 0
            for i in range(nb_of_iters):
                nodes = set(random.sample(nodes_in_decile, num_nodes)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            v = count / nb_of_iters
            if (count / nb_of_iters) > 0.5:
                high = middle
                v_high = v 
            else:
                low = middle
                v_low = v
        if abs(v_low - 0.5) < abs(v_high - 0.5):
            solutions_er_bc[d] = solutions_er_bc.get(d, 0) + low
        else:
            solutions_er_bc[d] = solutions_er_bc.get(d, 0) + high
            
for i in range(1, nb_decile+1):
    solutions_er_bc[i] = solutions_er_bc[i] / nb_of_graphs

In [ ]:
# Define your data
x_er = list(solutions_er_bc.keys())
y_er = list(solutions_er_bc.values())

# Define the trace for the bar plot
trace = go.Bar(x=x_er, y=y_er)

# Define the layout
layout = go.Layout(title='Impact of node degree on Metric Dimension: Analysis by quintiles', 
                   title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Metric Dimension'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

### Degree influence in Metric dimension of Barabasi-Albert graph 

In [ ]:
# Here we draw the nodes from the different quintile based on the degree of the nodes and 
# check if there are difference in the size of the resolving sets.

# We plot the size of the resolving set in a box plot

solutions_bar = {}
nb_of_graphs = 1
number_of_decile = 5
nb_of_iters = 100

# Barabasi parameters
n = 1000
m = 50

for g in tqdm(range(nb_of_graphs)):

    G = nx.barabasi_albert_graph(n, m)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    
    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []


    for d in range(1, number_of_decile+1): 
        sol = []
        nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == d]

        for _ in range(nb_of_iters):

            nodes = list(np.random.choice(nodes_in_decile, size=len(nodes_in_decile), replace=False)) # Random set of nodes to test
            for nb in range(0, len(nodes_in_decile)+1):
                if is_resolving_set(G, nodes[:nb], length):
                    sol.append(nb)
                    break
        solutions_bar[d] = solutions_bar.get(d, []) + sol

In [ ]:
fig = go.Figure()

for group, values in solutions_bar.items():
    fig.add_trace(go.Box(y=values, name=group))

    
fig.update_layout(
    yaxis=dict(
        title="Size of the resolving set",
        titlefont=dict(size=12, color='black')
    )
)
fig.show()

In [ ]:
# Here we find small resolving set and check the degree of the nodes

n = 100
m = 50
nb_of_iters = 10
nb_graphs = 1
lower_limit = 40

number_of_decile = 5

nb_from_each_decile = {}

for _ in tqdm(range(nb_graphs)):
    G = nx.barabasi_albert_graph(n, m)

    node_list = list(G.nodes())

    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_decile
    decile_num = 1
    decile_vertices = []

    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []
            
    length = dict(nx.all_pairs_shortest_path_length(G))

    for nb in range(lower_limit, G.number_of_nodes()): 
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                res = nodes
                break
        else:
            continue
        break
        
    second_elements = [t[1] for t in nx.get_node_attributes(G, 'decile').items() if t[0] in res]
    for d in second_elements:
        nb_from_each_decile[d] = nb_from_each_decile.get(d, 0) + 1

In [ ]:
total = sum(nb_from_each_decile.values())

nb_from_each_decile_in_perc = {}
for i in range(1, number_of_decile + 1):
    nb_from_each_decile_in_perc[i] = (nb_from_each_decile[i] / total) * 100

solution_bar = dict(sorted(nb_from_each_decile_in_perc.items()))

In [ ]:
# Define your data
x_er = list(solution_bar.keys())
y_er = list(solution_bar.values())

# Define the trace for the bar plot
trace = go.Bar(x=x_er, y=y_er)

# Define the layout
layout = go.Layout(#title='Impact of node degree on MD: Analysis by quintiles for Barabasi Albert Graph', 
                   #title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Percentage from each quintile'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [ ]:
# Here we draw the nodes from the different quintile and 
# check if there are difference in the size of the resolving sets.

solutions = {}
nb_of_graphs = 1
number_of_decile = 5

# Barabasi-Albert graph
n = 1000
m = 50
for g in tqdm(range(nb_of_graphs)):

    # Generate a Barabasi-Albert graph
    G = nx.barabasi_albert_graph(n, m)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    nb_of_iters = 100

    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []


    for d in range(1, number_of_decile+1):       
        nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == d]

        high = n//number_of_decile
        low = 0

        while low + 1 < high:
            middle = (high+low) // 2
            num_nodes = middle # Number of nodes to sample
            node_list = list(G.nodes())
            count = 0
            for i in range(nb_of_iters):
                nodes = set(random.sample(nodes_in_decile, num_nodes)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            v = count / nb_of_iters
            if (count / nb_of_iters) > 0.5:
                high = middle
                v_high = v 
            else:
                low = middle
                v_low = v
        if abs(v_low - 0.5) < abs(v_high - 0.5):
            solutions[d] = solutions.get(d, 0) + low
        else:
            solutions[d] = solutions.get(d, 0) + high
            
for i in range(1, number_of_decile+1):
    solutions[i] = solutions[i] / nb_of_graphs

In [ ]:
# Define your data
x = list(solutions.keys())
y = list(solutions.values())

# Define the trace for the bar plot
trace = go.Bar(x=x, y=y)

# Define the layout
layout = go.Layout(#title='Impact of node degree on Metric Dimension: Analysis by quintiles', 
                   #title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Metric Dimension'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

### Betweenness centrality influence in Metric dimension of Barabasi-Albert graph 

In [ ]:
# Here we find small resolving set and check the betweenness centrality of the nodes

nb_of_iters = 1000
nb_graphs = 1
lower_limit = 21
nb_decile = 5

# Barabasi graph parameters
n = 1000
m = 5

nb_from_each_decile = {}

for _ in range(nb_graphs):
    G = nx.barabasi_albert_graph(n, m)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    nb_of_iters = 100

    # Get all shortest paths between every pair of vertices
    shortest_paths = dict(nx.all_pairs_shortest_path(G))

    # Initialize a dictionary to store the counts of shortest paths that pass through each vertex
    path_counts = {v: 0 for v in G.nodes}

    # Count how many of the shortest paths pass through each vertex
    for source in G.nodes:
        for target in G.nodes:
            if source != target:
                paths = shortest_paths[source][target]
                for vertex in paths[1:-1]:
                    path_counts[vertex] += 1

    
    degree_list = [(n, d) for n, d in path_counts.items()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // nb_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []
    

    for nb in range(lower_limit, G.number_of_nodes()):
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                res = nodes
                break
        else:
            continue
        break
        
    second_elements = [t[1] for t in nx.get_node_attributes(G, 'decile').items() if t[0] in res]
    for d in second_elements:
        nb_from_each_decile[d] = nb_from_each_decile.get(d, 0) + 1

In [ ]:
total = sum(nb_from_each_decile.values())

nb_from_each_decile_in_perc = {}
for i in range(1, nb_decile+1):
    nb_from_each_decile_in_perc[i] = (nb_from_each_decile[i] / total) * 100

solution_bar = dict(sorted(nb_from_each_decile_in_perc.items()))

In [ ]:
# Define your data
x_er = list(solution_bar.keys())
y_er = list(solution_bar.values())

# Define the trace for the bar plot
trace = go.Bar(x=x_er, y=y_er)

# Define the layout
layout = go.Layout(title='Impact of node degree on Metric Dimension: Analysis by quintiles', 
                   title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Percentage of nodes'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

In [ ]:
# Here we draw the nodes from the different quintile based on betweenness and 
# check if there are difference in the size of the resolving sets.

nb_of_graphs = 1
nb_decile = 5

# barabasi_albert_graph
n = 1000
m = 50

solutions = {}
for g in range(nb_of_graphs):

    G = nx.barabasi_albert_graph(n, m)

    length = dict(nx.all_pairs_shortest_path_length(G))
    node_list = list(G.nodes())
    nb_of_iters = 100

    # Get all shortest paths between every pair of vertices
    shortest_paths = dict(nx.all_pairs_shortest_path(G))

    # Initialize a dictionary to store the counts of shortest paths that pass through each vertex
    path_counts = {v: 0 for v in G.nodes}

    # Count how many of the shortest paths pass through each vertex
    for source in G.nodes:
        for target in G.nodes:
            if source != target:
                paths = shortest_paths[source][target]
                for vertex in paths[1:-1]:
                    path_counts[vertex] += 1

    
    
    degree_list = [(n, d) for n, d in path_counts.items()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // nb_decile
    decile_num = 1
    decile_vertices = []
    
    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []


    for d in range(1, nb_decile+1):       
        nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == d]
        high = n // nb_decile
        low = 0

        while low + 1 < high:
            middle = (high+low) // 2
            num_nodes = middle # Number of nodes to sample
            node_list = list(G.nodes())
            count = 0
            for i in range(nb_of_iters):
                nodes = set(random.sample(nodes_in_decile, num_nodes)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            v = count / nb_of_iters
            if (count / nb_of_iters) > 0.5:
                high = middle
                v_high = v 
            else:
                low = middle
                v_low = v
        if abs(v_low - 0.5) < abs(v_high - 0.5):
            solutions[d] = solutions.get(d, 0) + low
        else:
            solutions[d] = solutions.get(d, 0) + high

In [ ]:
# Define your data
x = list(solutions.keys())
y = list(solutions.values())

# Define the trace for the bar plot
trace = go.Bar(x=x, y=y)

# Define the layout
layout = go.Layout(title='Impact of betweenness centrality on Metric Dimension: Analysis by quintiles', 
                   title_x=0.5,
                   xaxis=dict(title='Quintile'), 
                   yaxis=dict(title='Metric Dimension'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()